# Alert System For Detecting Obstacles

In [1]:
import torch, random
import torchvision
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import threading
import time
import playsound

In [4]:
# Recreate the model architecture
mobilenet = models.mobilenet_v2(pretrained=False)

# Modify the classifier to match the number of classes
num_classes = 10 
mobilenet.classifier[1] = nn.Linear(mobilenet.last_channel, num_classes)

mobilenet = mobilenet.to(device)

C:\Users\pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [6]:
mobilenet.load_state_dict(torch.load('obstacle_detection_mobilenet.pth', map_location=device))

C:\Users\pc\AppData\Local\Temp\ipykernel_24860\2737934849.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mobilenet.load_state_dict(torch.load('obstacle_detection_mobile

<All keys matched successfully>

In [ ]:
def visualize_prediction(model, img_path, true_class):
    model.eval()
    with torch.no_grad():
        # Prepare image for model
        input_img = img.unsqueeze(0)  # Add batch dimension
        output = model(input_img)
        pred_label = output.argmax(dim=1).item()  # Get the predicted class index
            
        # Display the image
        plt.imshow(img)
        plt.axis("off")
        plt.set_title(f"True: {idx_to_class[label]}\nPred: {idx_to_class[pred_label]}",
                fontsize=10, color="green" if label == pred_label else "red")
    
    plt.tight_layout()
    plt.show()

In [ ]:
def alert_obstacle():
    audio_file = 'alert.mp3'
    try:
        playsound.playsound(audio_file)
    except Exception as e:
        print(f"Error playing sound: {e}")

# def detect_real_time(model, data_loader, device, obstacle_class_index=1):
#     model.eval()
#     with torch.no_grad():
#         for inputs, labels in data_loader:
#             inputs = inputs.to(device)

#             outputs = model(inputs)
#             predictions = outputs.argmax(1)

#             if (predictions == obstacle_class_index).any():
#                 print("Obstacle detected!")
#                 threading.Thread(target=alert_obstacle).start()

#             time.sleep(0.5)


In [ ]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    return transform(image).unsqueeze(0)  # Add batch dimension

def detect_obstacle(image_paths, model, class_names):
    
    model.eval()
    for image_path in image_paths:
        try:
            # Load and preprocess the image
            image = load_image(image_path).to(device)

            # Make predictions
            with torch.no_grad():
                output = model(image)
                predicted_class = output.argmax(1).item()

            # Display results
            print(f"Image: {image_path}")
            print(f"Predicted Class: {class_names[predicted_class]}\n")
        except Exception as e:
            print(f"Error processing {image_path}: {e}")

In [ ]:
if __name__ == "__main__":
    # Load the fine-tuned model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    mobilenet = torchvision.models.mobilenet_v2(pretrained=False)
    mobilenet.classifier[1] = torch.nn.Linear(mobilenet.last_channel, num_classes)  # Update output layer
    mobilenet.load_state_dict(torch.load('obstacle_detection_mobilenet.pth', map_location=device))
    mobilenet.to(device)

    # Define the class names
    class_names = ['chair', 'door', 'fence', 'garbage_bin', 'obstacle', 'plant', 'pothole', 'stairs', 'table', 'vehicle']

    # List of image paths
    image_paths = [
        r'obstacles dataset/test/obstacle/gambar (239).jpg',
        r'obstacles dataset/test/obstacle/1691274417850139_5_jpg.rf.0e53c6a8542eb04effb104a33b470098.jpg',
        r'obstacles dataset/test/obstacle/34_jpg.rf.0a6c7459a48a7071c6c22f15f7a3c75c.jpg',
    ]

    # Detect obstacles
    detect_obstacle(image_paths, mobilenet, device, class_names)

In [ ]:
print("Obstacle detected!")
threading.Thread(target=alert_obstacle).start()